## HipHop Review Scraper ##

Set Up Imports

In [1]:
import requests
from bs4 import BeautifulSoup

Scrape Reviews from Pitchfork 

In [2]:
url = "https://pitchfork.com/reviews/albums/?genre=rap&page=1"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

Parse the soup for review links

In [3]:
links = []

# Parameter to make smarter later
link_size = 30
for link in soup.find_all('a'):
    if 'album' in str(link.get('href')) and len(str(link.get('href'))) > link_size:
        links.append(link.get('href'))

print(len(links))

12


Generate full URL's of the album reviews

In [4]:
base_url = "https://pitchfork.com"
review_urls = [base_url + i for i in links]
print(len(review_urls))

12


In [5]:
review_soup = []
for review_links in review_urls:
        response = requests.get(review_links)
        review_soup.append(BeautifulSoup(response.content, "html.parser"))

In [6]:
# Create a dictionary that maps tuple of name,album -> review paragraphs
album_reviews = {}
artist_class = "single-album-tombstone"
album_class = "single-album-tombstone__review-title"
content_class = "contents dropcap"

for i in range(len(review_soup)):
    # scrap the artist name
    artist_name = review_soup[i].find("div", {"class": artist_class}).findAll('a')[0].text
    # scrap the album name
    album_name = review_soup[i].find('h1', {"class": album_class}).text
    review_info = (artist_name,album_name)
    
    # scrap the review text
    r_paragraphs = review_soup[i].find("div",{"class": content_class}).findAll('p')
    review_text = ""
    for p in r_paragraphs:
        review_text += p.text
    
    album_reviews[review_info] = review_text

{('YBN Cordae', 'The Lost Boy'): 'The YBN Crew’s origin story is already the stuff of internet legend, even though the group formed just five years ago. Nick Simmons was born in Birmingham and met Jay Bradley, of Galveston, while playing Grand Theft Auto V. They bonded out of a shared love of rap music and live-streaming video games, eventually recording  their first freestyles in XBOX Live group chats as YBN Nahmir and YBN Almighty Jay, respectively.In 2017, Nahmir and YBN Glizzy met Cordae Amari Dunston online, and Dunston immediately became the elder statesman of the crew. While the group found viral success with Nahmir’s “Rubbin Off the Paint,” the vision they presented on last year’s YBN: the Mixtape was that of a three-headed monster led by Cordae.With The Lost Boy, Cordae makes good on the promise hinted at on YBN: The Mixtape, turning in a front-to-back redemption tale. It is Cordae at his most humble. It’s occasionally corny and self-serious. But its 15 tracks show a star slow

# Tokenize the Album Reviews #

In [30]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentences = {}

for key in album_reviews.keys():
    sentences[key] = album_reviews[key].split('.')

In [52]:
sentiment = {}
sia = SentimentIntensityAnalyzer()
# Aggregate the sentiment analysis scores
for keys in sentences:
    sent_scores = [0]*3
    for sentence in sentences[keys]:
        polarity = sia.polarity_scores(sentence)
        for i in range(len(sent_scores)):
            key = list(polarity.keys())[i]
            sent_scores[i] += polarity[key] 
    sentiment[keys] = sent_scores

# Normalize by number of sentences
for keys in sentiment:
    for i in range(len(sentiment[keys])):
        sentiment[keys][i] = sentiment[keys][i]/len(sentences[keys])
# Normalize by sum
for keys in sentiment:
    for i in range(len(sentiment[keys])):
        sentiment[keys][i] = sentiment[keys][i]/sum(sentiment[keys])
pos_neg_ratio = {}
for keys in sentiment:
    pos_neg_ratio[keys] = sentiment[keys][2]/sentiment[keys][0]
        
for keys in sentiment:
    print(keys,pos_neg_ratio[keys])

('YBN Cordae', 'The Lost Boy') 5.016552756673016
('Caleb Giles', 'Under the Shade') 2.0945741761230217
('Iggy Azalea', 'In My Defense') 0.9142175771011274
('Nas', 'The Lost Tapes 2') 0.3774550391771013
('Mase', 'Harlem World') 0.7710158119664865
('Maxo Kream', 'Brandon Banks') 1.6596604089096567
('Big K.R.I.T.', 'K.R.I.T. IZ HERE') 1.4721992785525269
('Key!', 'SO EMOTIONAL') 5.299431891646412
('Jaden Smith', 'ERYS') 1.9135559398960265
('Mr. Muthafuckin’ eXquire', 'Mr. Muthafuckin’ eXquire') 1.0691690787199537
('J Balvin', 'OASIS') 0.973144429516076
('Freddie Gibbs', 'Bandana') 1.710006409441644
